In [1]:
pip install geopy

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import geopy.distance as GPD
import pickle

# Data Acquisition
      
Found some Data at https://gis-kingcounty.opendata.arcgis.com/    
Data has lat and long for various types of places in King County including places of education, police/fire stations, airports and places for public gathering.    
Though it had the data available to download, the download always failed (it was BIG).
There were other ways with their own issues, I don't think poeple often want the data for the whole of King County. The way that seemed to work best is carried out below.     
Unsure of how to change the starting point, I didn't have a way to make a second query after the first exceeded the transfer limit. I felt that an incomplete listing was worse than none so those I dropped.

In [20]:
lookup = pd.read_csv('data/sitetype_lookup.csv', index_col='value')
lookup.head(12)

,site
value,
98,Other
99,No Gate No See
A1,Accessory Building
A2,Abandoned
A3,Group Mailbox
A9,Access Point
B1,Bridge
B2,Airport
B3,Helipad


In [5]:
lookup.loc['98','site']

'Other'

In [6]:
resp = requests.get("https://gisdata.kingcounty.gov/arcgis/rest/services/OpenDataPortal/property__parcel_address_area/MapServer/1722/query?where=SITETYPE%20%3D%20'R1'&outFields=SITETYPE,LAT,LON,ZIP5,PLUS4&outSR=4326&f=json")

In [25]:
x = json.loads(resp.text)

In [8]:
'exceededTransferLimit' not in x

False

In [9]:
x['features'][0]['attributes']

{'SITETYPE': 'R1',
 'LAT': 47.65611332,
 'LON': -122.35805293,
 'ZIP5': '98107',
 'PLUS4': '4928'}

In [12]:
data_lst = []
for value in lookup.index:
    resp = requests.get("https://gisdata.kingcounty.gov/arcgis/rest/services/OpenDataPortal/property__parcel_address_area/MapServer/1722/query?where=SITETYPE%20%3D%20'"+value+"'&outFields=SITETYPE,LAT,LON,ZIP5,PLUS4&outSR=4326&f=json")
    x = json.loads(resp.text)
    if 'exceededTransferLimit' not in x:
        for y in x['features']:
            entry = {}
            entry['sitetype'] = lookup.loc[value,'site'].replace(' ','_')
            entry['lat'] = y['attributes']['LAT']
            entry['lon'] = y['attributes']['LON']
            entry['zip'] = y['attributes']['ZIP5']
            entry['plus4'] = y['attributes']['PLUS4']
            data_lst.append(entry)
        

In [13]:
data = pd.DataFrame(data_lst)
data.groupby('sitetype').count()

,lat,lon,zip,plus4
sitetype,,,,
Abandoned,18,18,18,15
Access_Point,9,9,9,6
Accessory_Building,395,395,395,311
Airport,10,10,10,9
Camp/Bungalow,16,16,16,10
Campground,10,10,10,7
Cemetery,54,54,54,42
Commercial_Farm,75,75,75,67
Cultural,125,125,125,118


Some sitetypes have too few entries, like Group_Mailbox, or are to unclear like other; I'll drop these:

In [14]:
to_drop = ['Group_Mailbox','No_Gate_No_See', 'Other', 'Other_Residential', 
           'Town_Boundary_Point', 'Accessory_Building', 'Camp/Bungalow', 
           'PSAP','Hanger']
keep = list(set(data.sitetype) - set(to_drop))

In [15]:
data = data[data.sitetype.isin(keep)]

In [16]:
data.to_csv('gis.csv', index=False)

In [17]:
clean = pd.read_csv('data/clean.csv')
clean.head()

,date,price,bedrooms,sqft_living,sqft_lot,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bathroomsx4,floorsx2
0,164,221900.0,3,1180,5650,0,0,3,7,1180,0,1955,1955,98178,47.5112,-122.257,1340,5650,4,2
1,221,538000.0,3,2570,7242,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9,4
2,299,180000.0,2,770,10000,0,0,3,6,770,0,1933,1933,98028,47.7379,-122.233,2720,8062,4,2
3,221,604000.0,4,1960,5000,0,0,5,7,1050,910,1965,1965,98136,47.5208,-122.393,1360,5000,12,2
4,292,510000.0,3,1680,8080,0,0,3,8,1680,0,1987,1987,98074,47.6168,-122.045,1800,7503,8,2


In [23]:
data

,sitetype,lat,lon,zip,plus4
988,Abandoned,47.657762,-121.909312,98014,6321
989,Abandoned,47.523178,-121.927341,98027,None
990,Abandoned,47.529621,-122.164212,98059,3214
991,Abandoned,47.458155,-122.337274,98166,3019
992,Abandoned,47.192230,-121.993525,98022,9470
...,...,...,...,...,...
4935,Utility,47.363479,-122.246452,98032,7270
4936,Utility,47.365357,-122.239844,98032,7379
4937,Utility,47.492411,-122.268456,98178,3432
4938,Utility,47.478652,-122.261931,98168,4721


In [24]:
clean

,date,price,bedrooms,sqft_living,sqft_lot,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bathroomsx4,floorsx2
0,164,221900.0,3,1180,5650,0,0,3,7,1180,0,1955,1955,98178,47.5112,-122.257,1340,5650,4,2
1,221,538000.0,3,2570,7242,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9,4
2,299,180000.0,2,770,10000,0,0,3,6,770,0,1933,1933,98028,47.7379,-122.233,2720,8062,4,2
3,221,604000.0,4,1960,5000,0,0,5,7,1050,910,1965,1965,98136,47.5208,-122.393,1360,5000,12,2
4,292,510000.0,3,1680,8080,0,0,3,8,1680,0,1987,1987,98074,47.6168,-122.045,1800,7503,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21229,19,360000.0,3,1530,1131,0,0,3,8,1530,0,2009,2009,98103,47.6993,-122.346,1530,1509,10,6
21230,297,400000.0,4,2310,5813,0,0,3,8,2310,0,2014,2014,98146,47.5107,-122.362,1830,7200,10,4
21231,52,402101.0,2,1020,1350,0,0,3,7,1020,0,2009,2009,98144,47.5944,-122.299,1020,2007,3,4
21232,259,400000.0,3,1600,2388,0,0,3,8,1600,0,2004,2004,98027,47.5345,-122.069,1410,1287,10,4


Now I have this data I need to use it in some way, make it compatible with the data provided. In the code below, for each row in clean I take its lat and long and for each sitetype in keep, use geopy to measure the distance between it and each site of that type, find the minimum and record this.     
I also printed a number for each row, so I could see it was still running, and where it was up to.

In [ ]:
#Note code below takes a looooooooooooooooooooooooooooooong time to run


row_dists = []
for e, row in enumerate(clean.iterrows()):
    print(e)
    lat1 = row[1]['lat']
    lon1 = row[1]['long']
    coords1 = (lat1, lon1)
    dist_cols = {}
    for st in keep:
        df = data[data['sitetype'] == st]
        dists = []
        for site in df.iterrows():
            lat2 = site[1].lat
            lon2 = site[1].lon
            coords2 = (lat2, lon2)
            dist = GPD.geodesic(coords1, coords2).km
            dists.append(dist)
        dist_cols[st] = min(dists)
    row_dists.append(dist_cols)

In [ ]:
rd = pd.DataFrame(row_dists)
rd.rename(lambda col: col.replace('/', ''), axis=1, inplace=True)
#rd.to_csv('data/distances.csv', index=False)

Note:     
**Access Points**:   Regional Access Points (RAPs) are an entry point to CEA. These entry points are resource centers where households experiencing homelessness can get help finding housing and other resources. Individuals and families experiencing homelessness may call ahead to schedule an appointment.    
**PSAP**:    A public-safety answering point (PSAP), sometimes called "public-safety access point" is a call center where emergency calls (like police, fire brigade, ambulance) initiated by any mobile or landline subscriber are terminated.